In [2]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
from keras import Model
from keras.models import load_model
import numpy as np
import os
import pandas as pd

2021-11-18 14:20:46.401936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 14:20:46.402017: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [52]:
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'non_gene_2':'./data/gene/non_gene_2.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed',
    'ATAC_Bcell':'./data/gene/ATAC_Bcell.bed',
    'ATAC_Bcell':'./data/gene/ATAC_Bcell.bed',
    'ATAC_Brain':'./data/gene/ATAC_Brain.bed'
    }

data_y = []
data_x = []
with open(dic['ATAC_Bcell'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3'] and ll[2] == '5':
            data_x.append(TSS(ll[0],int(ll[1])))
            data_y.append(2)
num = len(data_x)
for i in range(5):
    count = 0
    with open(dic['ATAC_Bcell'],'r') as f:
        for line in f:
            ll = line.strip().split('\t')
            if ll[0] in ['1','2','3'] and ll[2] == str(i):
                data_x.append(TSS(ll[0],int(ll[1])))
                if i == 0 or i == 1:
                    data_y.append(0)
                elif i == 2 or i == 3:
                    data_y.append(1)
                elif i == 4:
                    data_y.append(2)
                count += 1
                if count > num:
                    break

bamfile = ps.AlignmentFile(dic['bam123'],'rb')

TSSes_x = data_x
perm = random.sample(range(len(data_x)),len(data_x))
TSSes_x = np.array(data_x)
TSSes_y = np.array(data_y)
TSSes_x = TSSes_x[perm[:len(perm)]]
TSSes_y = TSSes_y[perm[:len(perm)]]
labels = TSSes_y
up = 1000
down = 1000

In [53]:
print(max(labels))

2


In [54]:
'''lstm_input'''

feature_matrix = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            # ss = max(0, r.reference_start - start)
            # ee = min(r.reference_start - start + abs(r.isize), end - start)
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        wps = 0
        for r in bamfile.fetch(chrom[-1], ss+start, ee+start):
            if r.reference_start <= ss+start and r.reference_start+abs(r.isize) >= ee + start:
                wps += 1
            elif ee + start > r.reference_start > ss + start:
                wps -= 1
            elif ss+start < r.reference_start + abs(r.isize) < ee + start:
                wps -= 1
        ff.append(int(wps))
        feature_win[k] = np.array(ff)
        k =  k + 1
    feature_matrix.append(feature_win)
feature_matrix = np.array(feature_matrix)

lstm_x = []
for mat in feature_matrix:
    lstm_x.append(mat)
lstm_x = np.array(lstm_x)


In [55]:
print(lstm_x.shape)

(1289, 50, 4)


In [56]:
'''cnn_input'''
raw_data = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
#    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
    distribution_matrix = np.zeros((200,200),dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//10,relative_isize] += 1
    raw_data.append(distribution_matrix)
raw_data = np.array(raw_data)

cnn_x = []
for mat in raw_data:
    cnn_x.append(mat)
cnn_x = np.array(cnn_x)

In [66]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout, AveragePooling1D
from keras.models import Model
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
LL = keras.utils.to_categorical(labels)

cnn_input = Input(shape=(cnn_x.shape[1], cnn_x.shape[2]), name='cnn_input')
lstm_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
conv1 = Convolution1D(filters=64, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
# pool1 = AveragePooling1D(pool_size=2, strides=2)(conv1)
conv2 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(conv1)
cnn_output = Model(inputs=cnn_input, outputs=conv2)
main_input = concatenate([cnn_output.output, lstm_input])

lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(main_input)
conv = Convolution1D(filters=100, kernel_size=3, activation='relu',strides=1,padding='same',name='1')(lstm_out)
pool = MaxPooling1D(pool_size=2, strides=2,name='2')(conv)
drop = Dropout(0.2)(pool)
flatten = Flatten()(drop)
dense = Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4')(flatten)
drop = Dropout(0.2)(dense)
main_output = Dense(3, activation='softmax', kernel_regularizer=None, bias_regularizer=None,name='6')(drop)

model = Model(inputs=[cnn_output.input, lstm_input], outputs=main_output)

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], LL, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)


/home/jiay/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/300
10/10 - 3s - loss: 1.1772 - accuracy: 0.3371 - val_loss: 1.0124 - val_accuracy: 0.5581
Epoch 2/300
10/10 - 1s - loss: 1.0214 - accuracy: 0.4974 - val_loss: 0.9549 - val_accuracy: 0.5504
Epoch 3/300
10/10 - 1s - loss: 0.9533 - accuracy: 0.5224 - val_loss: 0.8948 - val_accuracy: 0.6047
Epoch 4/300
10/10 - 1s - loss: 0.9042 - accuracy: 0.5629 - val_loss: 0.8901 - val_accuracy: 0.5581
Epoch 5/300
10/10 - 1s - loss: 0.8784 - accuracy: 0.5603 - val_loss: 0.8885 - val_accuracy: 0.5891
Epoch 6/300
10/10 - 1s - loss: 0.8697 - accuracy: 0.5776 - val_loss: 0.8790 - val_accuracy: 0.5814
Epoch 7/300
10/10 - 1s - loss: 0.8548 - accuracy: 0.5853 - val_loss: 0.9054 - val_accuracy: 0.5736
Epoch 8/300
10/10 - 1s - loss: 0.8560 - accuracy: 0.5836 - val_loss: 0.8928 - val_accuracy: 0.5659
Epoch 9/300
10/10 - 1s - loss: 0.8463 - accuracy: 0.5931 - val_loss: 0.8867 - val_accuracy: 0.5969
Epoch 10/300
10/10 - 1s - loss: 0.8449 - accuracy: 0.6009 - val_loss: 0.8860 - val_accuracy: 0.5736
Epoch 11/

In [9]:
print(lstm_x.shape)

(1638, 50, 4)


In [19]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout, AveragePooling1D
from keras.models import Model
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
LL = keras.utils.to_categorical(labels)

cnn_input = Input(shape=(cnn_x.shape[1], cnn_x.shape[2]), name='cnn_input')
lstm_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
pool1 = AveragePooling1D(pool_size=2, strides=2)(conv1)
cnn_output = Model(inputs=cnn_input, outputs=pool1)
main_input = concatenate([cnn_output.output, lstm_input])
lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(main_input)
conv = Convolution1D(filters=100, kernel_size=3, activation='relu',strides=1,padding='same',name='1')(lstm_out)
pool = MaxPooling1D(pool_size=2, strides=2,name='2')(conv)
drop = Dropout(0.2)(pool)
flatten = Flatten()(drop)
dense = Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4')(flatten)
drop = Dropout(0.2)(dense)
main_output = Dense(2, activation='softmax', kernel_regularizer=None, bias_regularizer=None,name='6')(drop)

model = Model(inputs=[cnn_output.input, lstm_input], outputs=main_output)

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], LL, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/300
20/20 - 5s - loss: 0.5656 - accuracy: 0.7589 - val_loss: 0.4424 - val_accuracy: 0.8152
Epoch 2/300
20/20 - 2s - loss: 0.3783 - accuracy: 0.8485 - val_loss: 0.4299 - val_accuracy: 0.8152
Epoch 3/300
20/20 - 2s - loss: 0.3386 - accuracy: 0.8599 - val_loss: 0.4069 - val_accuracy: 0.8261
Epoch 4/300
20/20 - 2s - loss: 0.3145 - accuracy: 0.8675 - val_loss: 0.3876 - val_accuracy: 0.8297
Epoch 5/300
20/20 - 2s - loss: 0.3034 - accuracy: 0.8764 - val_loss: 0.3690 - val_accuracy: 0.8370
Epoch 6/300
20/20 - 1s - loss: 0.3006 - accuracy: 0.8792 - val_loss: 0.3593 - val_accuracy: 0.8370
Epoch 7/300
20/20 - 1s - loss: 0.2976 - accuracy: 0.8768 - val_loss: 0.4104 - val_accuracy: 0.8333
Epoch 8/300
20/20 - 1s - loss: 0.2886 - accuracy: 0.8857 - val_loss: 0.3680 - val_accuracy: 0.8442
Epoch 9/300
20/20 - 1s - loss: 0.2776 - accuracy: 0.8901 - val_loss: 0.3478 - val_accuracy: 0.8370
Epoch 10/300
20/20 - 2s - loss: 0.2718 - accuracy: 0.8914 - val_loss: 0.3402 - val_accuracy: 0.8370
Epoch 11/

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
LL = keras.utils.to_categorical(labels)
def attention_3d_block(inputs):
    a = Permute((2, 1))(inputs)
    a = Dense(50, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = layers.multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul
def get_attention_model(cnn_x, lstm_x):
    cnn_input = Input(shape=(cnn_x.shape[1], cnn_x.shape[2]), name='cnn_input')
    lstm_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
    conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
    pool1 = AveragePooling1D(pool_size=2, strides=2)(conv1)
    cnn_output = Model(inputs=cnn_input, outputs=pool1)
    main_input = concatenate([cnn_output.output, lstm_input])

    lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(main_input)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    main_output = Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(attention_mul)
    model = Model(inputs=[cnn_output.input, lstm_input], outputs=main_output)
    return model
model = get_attention_model(cnn_x,lstm_x)
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], labels, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/300
12/12 - 5s - loss: 0.6921 - accuracy: 0.5034 - val_loss: 0.6902 - val_accuracy: 0.4939
Epoch 2/300


/home/jiay/.local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


12/12 - 1s - loss: 0.6886 - accuracy: 0.6350 - val_loss: 0.6865 - val_accuracy: 0.7683
Epoch 3/300
12/12 - 1s - loss: 0.6849 - accuracy: 0.8446 - val_loss: 0.6827 - val_accuracy: 0.8659
Epoch 4/300
12/12 - 1s - loss: 0.6811 - accuracy: 0.8982 - val_loss: 0.6786 - val_accuracy: 0.9146
Epoch 5/300
12/12 - 1s - loss: 0.6768 - accuracy: 0.9043 - val_loss: 0.6742 - val_accuracy: 0.9024
Epoch 6/300
12/12 - 1s - loss: 0.6717 - accuracy: 0.9037 - val_loss: 0.6688 - val_accuracy: 0.9207
Epoch 7/300
12/12 - 1s - loss: 0.6657 - accuracy: 0.9057 - val_loss: 0.6623 - val_accuracy: 0.9207
Epoch 8/300
12/12 - 1s - loss: 0.6584 - accuracy: 0.9057 - val_loss: 0.6544 - val_accuracy: 0.9268
Epoch 9/300
12/12 - 1s - loss: 0.6495 - accuracy: 0.9084 - val_loss: 0.6450 - val_accuracy: 0.9207
Epoch 10/300
12/12 - 1s - loss: 0.6388 - accuracy: 0.9104 - val_loss: 0.6340 - val_accuracy: 0.9146
Epoch 11/300
12/12 - 1s - loss: 0.6268 - accuracy: 0.9091 - val_loss: 0.6213 - val_accuracy: 0.9146
Epoch 12/300
12/12 -

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
def attention_3d_block(inputs):
    a = Permute((2, 1))(inputs)
    a = Dense(50, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = layers.multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul
def get_attention_model(cnn_x, lstm_x):
    cnn_input = Input(shape=(cnn_x.shape[1], cnn_x.shape[2]), name='cnn_input')
    lstm_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
    conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
    pool1 = AveragePooling1D(pool_size=2, strides=2)(conv1)
    cnn_output = Model(inputs=cnn_input, outputs=pool1)
    main_input = concatenate([cnn_output.output, lstm_input])

    lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(main_input)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    main_output = Dense(2, activation='softmax', kernel_regularizer=None, bias_regularizer=None,name='6')(attention_mul)
    model = Model(inputs=[cnn_output.input, lstm_input], outputs=main_output)
    return model
model = get_attention_model(cnn_x,lstm_x)
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], LL, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/300
12/12 - 5s - loss: 0.6922 - accuracy: 0.4986 - val_loss: 0.6903 - val_accuracy: 0.5305
Epoch 2/300
12/12 - 1s - loss: 0.6899 - accuracy: 0.5536 - val_loss: 0.6881 - val_accuracy: 0.6341
Epoch 3/300
12/12 - 1s - loss: 0.6874 - accuracy: 0.7001 - val_loss: 0.6857 - val_accuracy: 0.7866
Epoch 4/300
12/12 - 1s - loss: 0.6846 - accuracy: 0.8033 - val_loss: 0.6828 - val_accuracy: 0.8354
Epoch 5/300
12/12 - 1s - loss: 0.6813 - accuracy: 0.8494 - val_loss: 0.6794 - val_accuracy: 0.8720
Epoch 6/300
12/12 - 1s - loss: 0.6774 - accuracy: 0.8820 - val_loss: 0.6752 - val_accuracy: 0.8963
Epoch 7/300
12/12 - 1s - loss: 0.6725 - accuracy: 0.8955 - val_loss: 0.6700 - val_accuracy: 0.8902
Epoch 8/300
12/12 - 1s - loss: 0.6665 - accuracy: 0.8982 - val_loss: 0.6636 - val_accuracy: 0.8963
Epoch 9/300
12/12 - 1s - loss: 0.6594 - accuracy: 0.9118 - val_loss: 0.6560 - val_accuracy: 0.9207
Epoch 10/300
12/12 - 1s - loss: 0.6505 - accuracy: 0.9091 - val_loss: 0.6464 - val_accuracy: 0.9207
Epoch 11/